In [1]:
import correctionlib
import correctionlib.convert
import uproot
import hist


In [2]:
inputdir = '/tmp/trigsf2l'

In [3]:
era_dict = {2015: '2016preVFP_UL', 2016: '2016postVFP_UL', 2017: '2017_UL', 2018: '2018_UL'}

name_dict = {
    'MuMu': 'h_DoubleMu_OR__X__allMET_mu0_pt_vs_mu1_pt_withSysts',
    'ElEl': 'h_DoubleEl_OR__X__allMET_el0_pt_vs_el1_pt_withSysts',
    'MuEl': 'h_EMu_OR__X__allMET_mu0_pt_vs_el0_pt_withSysts',
}


In [4]:
for era in era_dict.values():
    corrs = []
    for name, histname in name_dict.items():
        histpath = f'{inputdir}/{era}.root:{histname}'

        h = uproot.open(histpath)
        hn = hist.Hist(hist.axis.StrCategory(['central', 'up', 'down'], name='sf', label='sf type (central, up, down)'),
                       *h.to_boost().axes, name=f'{name}TriggerSF', label=f'{name}TriggerSF')
        hn['central', :, :] = h.values()
        hn['up', :, :] = h.values() + h.errors()
        hn['down', :, :] = h.values() - h.errors()

        axis_names = ['Leading lepton pT', 'Sub-leading lepton pT']
        if name == 'MuEl':
            axis_names = ['Muon pT', 'Electron pT']
        corr = correctionlib.convert.from_histogram(hn, ['sf'] + axis_names, flow='clamp')
        corr.name = f'{name}TriggerSF'
        corrs.append(corr)

    cset = correctionlib.schemav2.CorrectionSet(schema_version=2, corrections=corrs)

    with open(f"{era}_DiLeptonTriggerSF.json", "w") as fout:
        fout.write(cset.json(exclude_unset=True))
